# SQLAlchemy ORM with French Cities & Departments

This notebook demonstrates SQLAlchemy's Object-Relational Mapping (ORM) using a real dataset of French cities and departments.

## Database Connection Setup

We connect to a SQLite database containing French geographical data:
- **SQLite**: File-based database, perfect for learning and prototyping
- **echo=True**: Shows all SQL queries SQLAlchemy generates (educational!)
- **Engine**: SQLAlchemy's "factory" for database connections


In [ ]:
from sqlalchemy import create_engine
from pathlib import Path


p_departement = Path.cwd().parent.parent / "data" / "villes_france.db" 
engine = create_engine(f"sqlite:///{p_departement}", echo=True)

engine

## ORM Class Definitions

**Object-Relational Mapping (ORM)** lets us work with database tables as Python classes:

### Key Concepts:
- **DeclarativeBase**: All ORM classes inherit from this
- **Mapped[type]**: Type hints that SQLAlchemy understands
- **mapped_column()**: Defines database column properties
- **ForeignKey**: Creates relationships between tables

### Table Relationships:
- `Ville.department` → `Departement.departement_code` (many-to-one)
- Each city belongs to exactly one department
- Each department can have many cities


In [ ]:
from sqlalchemy import ForeignKey
from sqlalchemy import String, Text
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

class Base(DeclarativeBase):
    pass

class Departement(Base):
    __tablename__ = "departement"
    departement_id: Mapped[int] = mapped_column(primary_key=True)
    departement_code: Mapped[str] = mapped_column(Text)
    departement_nom: Mapped[str] = mapped_column(Text)

    def is_outremer(self) -> bool:
        """Custom method: Overseas departments have 3-digit codes (97x)"""
        return len(self.departement_code) == 3

class Ville(Base):
    __tablename__ = "villes"
    
    id: Mapped[int] = mapped_column(primary_key=True)  # AUTOINCREMENT handled automatically
    department: Mapped[str] = mapped_column(ForeignKey("departement.departement_code"))  # Add FK
    name: Mapped[str] = mapped_column(Text)
    population_2012: Mapped[int] = mapped_column()
    surface: Mapped[float] = mapped_column()  # REAL maps to float
    commune_code: Mapped[str] = mapped_column(Text, unique=True)
    
    def __repr__(self) -> str:
        return f"Ville(name={self.name}, department={self.department}, pop={self.population_2012})"



In [ ]:
from sqlalchemy.orm import Session
from sqlalchemy import select

session = Session(engine)


The **Session** is your main interface to the database:

- **Tracks objects**: Knows which objects are new, modified, or deleted
- **Manages transactions**: Groups operations together
- **Identity map**: Ensures one Python object per database row
- **Lazy loading**: Can fetch related data automatically

Think of it as your "conversation" with the database.

In [ ]:
stmt = (
    select(Departement)
    .where(Departement.departement_code.startswith("9")) # All the departemens starting with 6
)


This is **class-level attribute access**:

- `Departement.departement_code` → References the column definition in your class
- `.startswith("6")` → SQLAlchemy translates this to SQL: `WHERE departement_code LIKE '6%'`
- **No SQL strings needed!** Pure Python expressions

In [ ]:
ans = session.execute(stmt) # Returns Result object

for dept in ans.scalars():
    # dept is now a full Python object with all your class methods/attributes
    print(dept.departement_nom, dept.departement_code) # Access database columns, As Python attributes
    # If you had methods in your class, you could call them here too!
    print(f"The department {dept.departement_nom} is {'not ' if not dept.is_outremer() else ''}overseas.")

**What scalars() does:**

- **Input**: `Row(Departement(...))` - wrapped objects
- **Output**: `Departement(...)` - direct objects
- **Why**: Makes iteration cleaner and more Pythonic
The scalars() method unwraps these Row objects so you get your actual model objects directly:
Bottom line: scalars() removes the Row wrapper so you can work with your Python objects directly instead of having to use row[0] everywhere.

## Complex Query Example: Joins + Aggregations

Let's answer: **"Which overseas department has the most cities with population > 500?"**

### Query Building Blocks:

**1. SELECT clause:**
- `Departement` → Full department object  
- `func.count(Ville.id).label('total_cities')` → Count cities per department

**2. JOIN:**
- Links tables: `Ville.department == Departement.departement_code`
- Now we can filter/group across both tables

**3. WHERE clauses:**
- `Ville.population_2012 > 500` → Only substantial cities
- `Departement.departement_code.startswith("97")` → Only overseas (97x codes)

**4. Aggregation:**
- `group_by(Departement.departement_id)` → One result per department
- `func.count()` → Count cities within each group

**5. Ordering & Limiting:**
- `order_by(...desc())` → Highest count first
- `limit(1)` → Just the winner


In [ ]:
from sqlalchemy import func
#Let's say we want to know which overseas departement
# has the most cities of more than 500 inhabitants.
# why not...

stmt = (
    select(
        Departement,
        func.count(Ville.id).label('total_cities')
        )
    .join(Departement, Ville.department == Departement.departement_code)    
    .where(Ville.population_2012 > 500)
    .where(Departement.departement_code.startswith("97"))
    .group_by(Departement.departement_id)
    .order_by(func.count(Ville.id).desc())
    .limit(1)
    )

### Query Execution & Result Handling

When your SELECT includes multiple items, SQLAlchemy returns **tuples**:
- `(Departement_object, count_value)`
- Use tuple unpacking: `dept, count = row`

**Two patterns:**
- `fetchall()` → Returns list of all rows (good for iteration)
- `fetchone()` → Returns just the first row (good for single results)


In [8]:
results = session.execute(stmt).fetchall()

2025-08-29 22:56:29,376 INFO sqlalchemy.engine.Engine SELECT departement.departement_id, departement.departement_code, departement.departement_nom, count(villes.id) AS total_cities 
FROM villes JOIN departement ON villes.department = departement.departement_code 
WHERE villes.population_2012 > ? AND (departement.departement_code LIKE ? || '%') GROUP BY departement.departement_id ORDER BY count(villes.id) DESC
 LIMIT ? OFFSET ?
2025-08-29 22:56:29,376 INFO sqlalchemy.engine.Engine [generated in 0.00059s] (500, '97', 1, 0)


In [9]:
for row in results:
    dept, count = row  # Unpack the tuple
    print(f"Department: {dept.departement_nom}, Cities > 500: {count}")



Department: Guadeloupe, Cities > 500: 34


In [10]:
# Or just the first result since we know we do limit 1:
dept, count = session.execute(stmt).fetchone()
print(f"Winner: {dept.departement_nom} with {count} cities")

2025-08-29 22:56:30,779 INFO sqlalchemy.engine.Engine SELECT departement.departement_id, departement.departement_code, departement.departement_nom, count(villes.id) AS total_cities 
FROM villes JOIN departement ON villes.department = departement.departement_code 
WHERE villes.population_2012 > ? AND (departement.departement_code LIKE ? || '%') GROUP BY departement.departement_id ORDER BY count(villes.id) DESC
 LIMIT ? OFFSET ?
2025-08-29 22:56:30,779 INFO sqlalchemy.engine.Engine [cached since 1.404s ago] (500, '97', 1, 0)
Winner: Guadeloupe with 34 cities
